# FLAN-T5 laptops14 ABSA (Aspect + Polarity)

In [1]:
#%pip install --upgrade pip

In [2]:
#%pip install transformers

In [3]:
#%pip install accelerate -U

In [4]:
#%pip install datasets

In [5]:
#%pip install git+https://github.com/dask/s3fs

In [6]:
#%pip install wandb

In [7]:
#%pip install torch

In [8]:
import pandas as pd
import os
import warnings
from datasets import load_dataset
import json
import random
from transformers import AutoTokenizer
from datasets import concatenate_datasets
from transformers import AutoModelForSeq2SeqLM
from transformers import DataCollatorForSeq2Seq
from huggingface_hub import HfFolder
from transformers import Seq2SeqTrainer, Seq2SeqTrainingArguments
import torch
import wandb


warnings.filterwarnings("ignore")

/home/jupyter/.local/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
2024-03-12 18:34:32.561977: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [9]:
raw_datasets = load_dataset("alexcadillon/SemEval2014Task4", "laptops")
raw_datasets

DatasetDict({
    trial: Dataset({
        features: ['sentenceId', 'text', 'aspectTerms'],
        num_rows: 100
    })
    train: Dataset({
        features: ['sentenceId', 'text', 'aspectTerms'],
        num_rows: 3045
    })
    test: Dataset({
        features: ['sentenceId', 'text', 'aspectTerms'],
        num_rows: 800
    })
})

In [10]:
def preprocess_for_prompts(sample):
    with open('prompts_absa.json') as fp:
        template = json.load(fp)

    num = random.randint(0, len(template)-1)
    instruction =  template[str(num)]

    sample["aspect_polarities_list"] = ",".join([f"{item['term']}: {item['polarity']}" for item in sample['aspectTerms']])
    sample['aspect_polarities_output'] = f"Answer: \n{sample['aspect_polarities_list']}"
    sample["aspect_polarity_input"] = (
        f"{instruction}\n{sample['text']}\n"
    )
    return sample

In [11]:
augmented_dataset = raw_datasets.map(preprocess_for_prompts)
augmented_dataset['train'][10]

Parameter 'function'=<function preprocess_for_prompts at 0x7f2c95383c70> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/3045 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

{'sentenceId': '80',
 'text': 'Great laptop that offers many great features!',
 'aspectTerms': [{'term': 'features',
   'polarity': 'positive',
   'from': '36',
   'to': '44'}],
 'aspect_polarities_list': 'features: positive',
 'aspect_polarities_output': 'Answer: \nfeatures: positive',
 'aspect_polarity_input': 'Can you tell me the aspects (explicit and implicit) and their sentiment polarity of the sentence below. Thank you!\nGreat laptop that offers many great features!\n'}

In [12]:
model_id = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [13]:
# The maximum total input sequence length after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded.
tokenized_inputs = concatenate_datasets(
    [augmented_dataset["train"], augmented_dataset["test"], augmented_dataset["trial"]],
).map(
    lambda x: tokenizer(x["aspect_polarity_input"], truncation=True),
    batched=True,
    remove_columns=[
        "sentenceId",
        "text",
        "aspectTerms",
        "aspect_polarities_list",
        "aspect_polarities_output",
        "aspect_polarity_input",
    ],
)
max_source_length = max([len(x) for x in tokenized_inputs["input_ids"]])
print(f"Max source length: {max_source_length}")

# The maximum total sequence length for target text after tokenization.
# Sequences longer than this will be truncated, sequences shorter will be padded."
tokenized_targets = concatenate_datasets(
    [augmented_dataset["train"], augmented_dataset["test"], augmented_dataset["trial"]]
).map(
    lambda x: tokenizer(x["aspect_polarities_output"], truncation=True),
    batched=True,
    remove_columns=[
        "sentenceId",
        "text",
        "aspectTerms",
        "aspect_polarities_list",
        "aspect_polarities_output",
        "aspect_polarity_input",
    ],
)
max_target_length = max([len(x) for x in tokenized_targets["input_ids"]])
print(f"Max target length: {max_target_length}")

Map:   0%|          | 0/3945 [00:00<?, ? examples/s]

Max source length: 122


Map:   0%|          | 0/3945 [00:00<?, ? examples/s]

Max target length: 76


In [14]:
def preprocess_function(sample, padding="max_length"):

    model_inputs = tokenizer(
        sample["aspect_polarity_input"],
        max_length=max_source_length,
        padding=padding,
        truncation=True,
    )

    # Tokenize targets with the `text_target` keyword argument
    labels = tokenizer(
        text_target=sample["aspect_polarities_output"],
        max_length=max_target_length,
        padding=padding,
        truncation=True,
    )

    # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 when we want to ignore
    # padding in the loss.
    if padding == "max_length":
        labels["input_ids"] = [
            [(l if l != tokenizer.pad_token_id else -100) for l in label]
            for label in labels["input_ids"]
        ]

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


tokenized_dataset = augmented_dataset.map(
    preprocess_function,
    batched=True,
    remove_columns=[
        "sentenceId",
        "text",
        "aspectTerms",
        "aspect_polarities_list",
        "aspect_polarities_output",
        "aspect_polarity_input",
    ],
)
print(f"Keys of tokenized dataset: {list(tokenized_dataset['train'].features)}")

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/3045 [00:00<?, ? examples/s]

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Keys of tokenized dataset: ['input_ids', 'attention_mask', 'labels']


In [15]:
# huggingface hub model id
model_id = "google/flan-t5-base"

# load model from the hub
model = AutoModelForSeq2SeqLM.from_pretrained(model_id)

In [16]:
# we want to ignore tokenizer pad token in the loss
label_pad_token_id = -100
# Data collator
data_collator = DataCollatorForSeq2Seq(
    tokenizer, model=model, label_pad_token_id=label_pad_token_id, pad_to_multiple_of=8
)

In [17]:
from huggingface_hub import HfFolder
HfFolder.save_token(os.environ['hugging_face_login'])

In [18]:
os.environ["WANDB_PROJECT"] = "absa_research2"
os.environ["WANDB_LOG_MODEL"] = "true"
wandb.login(key=os.environ["wandb_login"])

wandb: Currently logged in as: katya_shakhova (shakhova). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/jupyter/.netrc


True

In [19]:
# Hugging Face repository id
repository_id = f"{model_id.split('/')[1]}-absa-laptops"

# Define training args
training_args = Seq2SeqTrainingArguments(
    output_dir=repository_id,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    predict_with_generate=True,
    fp16=False,  # Overflows with fp16
    learning_rate=5e-5,
    num_train_epochs=5,
    warmup_ratio=0.1,
    weight_decay=0.1,
    optim="adamw_torch",
    # logging & evaluation strategies
    evaluation_strategy="steps",
    eval_steps=100,
    report_to="wandb",
    logging_steps=100,
    save_strategy="no",
    save_total_limit=1,
    push_to_hub=False,
    hub_model_id=repository_id,
    hub_token=HfFolder.get_token(),
)

# Create Trainer instance
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],

)

In [20]:
trainer.train()

wandb: Tracking run with wandb version 0.16.4
wandb: Run data is saved locally in /home/jupyter/work/resources/wandb/run-20240312_183903-35fasi5i
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run breezy-frog-4
wandb: ⭐️ View project at https://wandb.ai/shakhova/absa_research2
wandb: 🚀 View run at https://wandb.ai/shakhova/absa_research2/runs/35fasi5i


Step,Training Loss,Validation Loss
100,2.482400,0.419161
200,0.507300,0.280843
300,0.383700,0.226439
400,0.329000,0.210451
500,0.291000,0.211129
600,0.249700,0.192658
700,0.236800,0.191988
800,0.206800,0.193867
900,0.230600,0.191607


TrainOutput(global_step=955, training_loss=0.526438941755844, metrics={'train_runtime': 219.3228, 'train_samples_per_second': 69.418, 'train_steps_per_second': 4.354, 'total_flos': 2606357672755200.0, 'train_loss': 0.526438941755844, 'epoch': 5.0})

In [21]:
# Save our tokenizer and create model card
tokenizer.save_pretrained(repository_id)
trainer.create_model_card()
# Push the results to the hub
trainer.push_to_hub()

training_args.bin:   0%|          | 0.00/4.67k [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Shakhovak/flan-t5-base-absa-laptops/commit/91c06dc35bd50ee91836718867ee762d6a3029f1', commit_message='End of training', commit_description='', oid='91c06dc35bd50ee91836718867ee762d6a3029f1', pr_url=None, pr_revision=None, pr_num=None)

In [22]:
wandb.finish()

wandb:                                                                                
wandb: 
wandb: Run history:
wandb:                      eval/loss █▄▂▂▂▁▁▁▁
wandb:                   eval/runtime █▂▄▄▅▁▁▁▂
wandb:        eval/samples_per_second ▁▇▅▅▄███▇
wandb:          eval/steps_per_second ▁▇▅▅▄███▇
wandb:                    train/epoch ▁▁▂▂▃▃▃▃▄▄▅▅▆▆▇▇███
wandb:              train/global_step ▁▁▂▂▃▃▃▃▄▄▅▅▆▆▇▇███
wandb:                train/grad_norm █▂▃▂▂▂▁▁▁
wandb:            train/learning_rate █▇▆▅▄▄▃▂▁
wandb:                     train/loss █▂▂▁▁▁▁▁▁
wandb:               train/total_flos ▁
wandb:               train/train_loss ▁
wandb:            train/train_runtime ▁
wandb: train/train_samples_per_second ▁
wandb:   train/train_steps_per_second ▁
wandb: 
wandb: Run summary:
wandb:                      eval/loss 0.19161
wandb:                   eval/runtime 2.9844
wandb:        eval/samples_per_second 268.064
wandb:          eval/steps_per_second 16.754
wandb:                 